In [1]:
import pandas as pd
import pickle 
import numpy as np
import datetime as dt

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

### Read the dataFile 
Read = pickle.load( open( "uke_eggedal_data_challenge.pkl", "rb" ) )

#save data on my variable
data  = {}
for ids,sets in Read.items():
    data[ids] = sets

#Check keys and data types in the  dic
for ids,sets in data.items():
    print(ids, f', Dtype = {type(sets)}, Total values in Set:{len(sets)}')
    
#Separate the obs_tas and obs_pr arrays into individual columns to make things easier to see
Temp = {}
precipitation ={}

#create dummy data for dictionary depending on lenght of values 
for i in data['obs_pr']:
    Length = len(i)
    for j in range (Length):
        precipitation[f'Prec_Zone_{j}'] = []
#Fill the empty list with the corresponding data 
index = 0
for ids,val in precipitation.items():
    for i in data['obs_pr']:
        precipitation[ids].insert(index,i[index])
    index = index+1    
dfprecipitation = pd.DataFrame(precipitation)

# Repeat for Temperature

#create dummy data for dictionary depending on lenght of values 
for i in data['obs_tas']:
    Length = len(i)
    for j in range (Length):
        Temp[f'Temp_Zone_{j}'] = []
#Fill the empty list with the corresponding data 
index = 0
for ids,val in Temp.items():
    for i in data['obs_tas']:
        Temp[ids].insert(index,i[index])
    index = index+1  
dfTemp = pd.DataFrame(Temp)
#add the date to each DF 
dfTemp['Date'] = data['obs_dates'].reset_index(drop=True) # Reset Index to keep same index as initial DF
dfTemp['Date'] = pd.to_datetime(dfTemp['Date'])
dfprecipitation['Date'] = data['obs_dates'].reset_index(drop=True) # Reset Index to keep same index as initial DF
dfprecipitation['Date'] = pd.to_datetime(dfprecipitation['Date'])
# Create new DataFrame for all data together
DfTemp_Prec = dfprecipitation.merge(dfTemp, how='inner', on='Date')

#Create DataFrame for de Flow data
dfFlow = pd.DataFrame(data['flow'],columns = ['Flow'])
dfFlow['Date'] = data['flow_dates'].reset_index(drop=True)
#Now we merge the two datasets where the dates are the same 
Df_Final= DfTemp_Prec.merge(dfFlow, how='inner', on='Date')

flow , Dtype = <class 'numpy.ndarray'>, Total values in Set:21542
obs_tas , Dtype = <class 'numpy.ndarray'>, Total values in Set:21546
flow_dates , Dtype = <class 'pandas.core.series.Series'>, Total values in Set:21542
obs_lon_lat , Dtype = <class 'list'>, Total values in Set:2
obs_pr , Dtype = <class 'numpy.ndarray'>, Total values in Set:21546
obs_dates , Dtype = <class 'pandas.core.series.Series'>, Total values in Set:21546


In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [3]:
# define input sequence
raw_seq = Df_Final['Flow']
# choose a number of time steps
n_steps = 20
# split into samples
X, y = split_sequence(raw_seq, n_steps)

In [4]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [5]:
#FinalTest = Df_Final[(Df_Final['Date'] >= f'2015-01-02')]
Original= DfTemp_Prec.merge(dfFlow, how='inner', on='Date')
Original = Original[(Original['Date'] >= f'2015-01-02')]

In [6]:
XFinal = pd.DataFrame(columns=['Flow'])
XFinal['Flow'] = Original['Flow'][:20]

In [7]:
# demonstrate prediction
x_input = np.array(XFinal['Flow'])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1.4755273]]


In [9]:
Original['Flow'][:21]

20814    1.522931
20815    0.364669
20816    2.190361
20817    2.729620
20818    1.018141
20819    2.727857
20820    1.709606
20821    0.107957
20822    2.686432
20823    1.372319
20824    4.118482
20825    3.260375
20826    2.173459
20827    2.351025
20828    1.312288
20829    0.132999
20830    1.909995
20831    3.441909
20832    1.721519
20833    0.001000
20834    1.465748
Name: Flow, dtype: float64

In [10]:
model.save('kerasModel')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: kerasModel\assets
